In [ ]:
from itertools import combinations

import random

In [ ]:
# Функция для генерации областей неразличимости для каждого игрока.
# На вход подаем карты игрока, общее количество карт в игре, количество карт у всех остальных игроков по порядку (от a до r).

def get_worlds(player_cards, n, b, c, r, num):
  all = [i+1 for i in range(n)]
  result = []

  for second_player in combinations(list(set(all) - set(player_cards)), b): # Перебираем всевозможные карты второго игрока
    for rival in combinations(list(set(all) - set(second_player) - set(player_cards)), c): # Перебираем всевозможные карты третьего игрока
      remaining = list(set(all) - set(player_cards) - set(second_player) - set(rival))
      if num == 0:
        result.append([sorted(player_cards), sorted(list(second_player)), sorted(list(rival)), sorted(remaining)])
      elif num == 1:
        result.append([sorted(list(second_player)), sorted(player_cards), sorted(list(rival)), sorted(remaining)])
      else:
        result.append([sorted(list(second_player)), sorted(list(rival)), sorted(player_cards), sorted(remaining)])

  return result


In [ ]:
# Функция для генерации всевозможных миров по входным данным
# На вход подаем общее количество карт в игре, количество карт у всех игроков по порядку от a до r.

# Механизм работы функции аналогичен предыдущей, за исключением того, что нет зафиксированного набора карт одного из игроков

def get_all_worlds(n, a, b, c, r):
  all = [i+1 for i in range(n)]
  result = []

  for first_player in combinations(all, a):
    for second_player in combinations(list(set(all) - set(first_player)), b):
      for rival in combinations(list(set(all) - set(second_player) - set(first_player)), c):
        remaining = list(set(all) - set(first_player) - set(second_player) - set(rival))
        result.append([sorted(list(first_player)), sorted(list(second_player)), sorted(list(rival)), sorted(remaining)])
  return result

In [ ]:
# Функция, проверяющая, что в каждом из миров в множестве неразличимости игрока X содержатся одинаковые карты другого игрока Y.
# Это говорит о том, что X знает карты игрока Y

def check_all_same(worlds, player_num, checker):
  for world in worlds:
    if world[player_num] != checker:
      return 0
  return 1

In [ ]:
# Функции определяющие победителя, а также регулирующие продолжение движения по циклу.

def winAB(A_suggested, B_suggested, A_real, B_real):
  if check_all_same(A_suggested, 1, B_real) or check_all_same(B_suggested, 0, A_real): # Победа игроков A и B
    return 1
  return 0

def winC(C_suggested, A_real, B_real):
  if check_all_same(C_suggested, 0, A_real) or check_all_same(C_suggested, 1, B_real): # Победа игрока C
    return 1
  return 0

In [ ]:
def turn(all, A, B, C, num):
  cur = random.choice(all[num]) # Выбираем рандомный мир из базы (реальный оттуда удалили изначально)
  if cur in A:
    A.remove(cur) # Если A считает мир cur возможным, удаляем его
  if cur in B:
    B.remove(cur) # Если B считает мир cur возможным, удаляем его
  if cur in C:
    C.remove(cur) # Если C считает мир cur возможным, удаляем его

  if cur in all[0]:
    all[0].remove(cur) # Удаляем мир из общей базы миров A
  if cur in all[1]:
    all[1].remove(cur) # Удаляем мир из общей базы миров B
  if cur in all[2]:
    all[2].remove(cur) # Удаляем мир из общей базы миров C

In [ ]:
# Простая функция для удаления предполагаемых миров из общей базы

def delete_suggested(all, suggested):
  for world in suggested:
    all.remove(world)
  return all

In [ ]:
def game(n, a, b, c, r):
  real_world = [i+1 for i in range(n)]
  random.shuffle(real_world) # Получаем распределение карт в реальном мире случайным образом.

  # Получаем карты у каждого игрока на руках по отдельности.
  A_real = sorted(real_world[:a])
  B_real = sorted(real_world[a:a+b])
  C_real = sorted(real_world[a+b:a+b+c])
  R_real = sorted(real_world[a+b+c:])
  tmp = [A_real, B_real, C_real, R_real] # Формируем реальный мир вида [[A], [B], [C], [R]], так как все группы миров в дальнейшем будут в таком же формате

  # Получаем множества неразличимых миров для каждого игрока по отдельности.
  A_suggested = get_worlds(A_real, n, b, c, r, 0)
  B_suggested = get_worlds(B_real, n, a, c, r, 1)
  C_suggested = get_worlds(C_real, n, a, b, r, 2)

  # Получаем весь список возможных миров при входных данных n, a, b, c, r
  # и удаляем из общего списка те миры, которые агенты не стали бы удалять.
  all_worlds_A_tmp = get_all_worlds(n, a, b, c, r)
  all_worlds_B_tmp = get_all_worlds(n, a, b, c, r)
  all_worlds_C_tmp = get_all_worlds(n, a, b, c, r)

  all_worlds_A = delete_suggested(all_worlds_A_tmp, A_suggested)
  all_worlds_B = delete_suggested(all_worlds_B_tmp, B_suggested)
  all_worlds_C = delete_suggested(all_worlds_C_tmp, C_suggested)

  # Продолжаем ходы до тех пор, пока кто-то из агентов не победит или мы не получим ничью.
  while not winAB(A_suggested, B_suggested, A_real, B_real) and not winC(C_suggested, A_real, B_real):
    if len(all_worlds_A) == 0:
      break
    turn([all_worlds_A, all_worlds_B, all_worlds_C], A_suggested, B_suggested, C_suggested, 0)
    if len(all_worlds_B) == 0:
      break
    turn([all_worlds_A, all_worlds_B, all_worlds_C], A_suggested, B_suggested, C_suggested, 1)
    if len(all_worlds_C) == 0:
      break
    turn([all_worlds_A, all_worlds_B, all_worlds_C], A_suggested, B_suggested, C_suggested, 2)

  # По результату функции win можем однозначно определить победителя.
  if winAB(A_suggested, B_suggested, A_real, B_real) and not winC(C_suggested, A_real, B_real):
    return 1
  else:
    return 0

In [ ]:
def generate_m_games():
  print("Введите количество игр: ")
  m = int(input())
  print("Введите входные данные в формате n a b c r через пробел: ")
  n, a, b, c, r = map(int, input().split())
  result = 0

  for i in range(m):
    result += game(n, a, b, c, r)

  return result


In [ ]:
generate_m_games()

Введите количество игр: 
1000
Введите входные данные в формате n a b c r через пробел: 
8 3 3 1 1


866